## Умова лабораторної роботи

Дано масив $T$, який складається з $N$ рядків, які відповідають прикладам, і $m$ стовпчиків, які відповідають ознакам. Відомо, що ознака $x_h$ приймає значення з множини $\{c_{h,1},\ c_{h,2},\ \ldots,\ c_{h,q_h}\}$. Дано вектор $y$ розмірності $N$, елементи якого приймають значення з множини $S=\{s_1,\ s_2,\ \ldots,\ s_{v}\}$ (мітки класів для прикладів). Знайти ознаку $x^{*}_{h}$, для якої наступний вираз приймає мінімальне значення:
$$G(x_{h})=\sum\limits_{i=1}^{q_{h}}\dfrac{|T_{i}|}{N}H(T_{i},\ S)$$
де $T_{i}$ - підмножина прикладів, для яких ознака $x_{h}$ приймає значення $c_{hi}$, $|A|$ - потужність множини $A$, $H(A,\ S)$ - ентропія множини $A$ по відношенню до властивості $S$:
$$H(A,\ S)=-\sum\limits_{i=1}^{v}\dfrac{k_{i}}{|A|}\log_2\dfrac{k_{i}}{|A|},$$
де властивість $S$ може приймати $v$ різних значень, кожне з яких - в $k_{i}$ випадках.

## Імпорт бібліотек та функції

In [1]:
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import LabelEncoder
from scipy.stats import entropy

## Програмна реалізація

Код функцій

In [ ]:
def Entropy(prob):
    """
    Обчислює ентропію для підмножини міток. 
    Parameters:
        subset_labels (numpy.ndarray): масив ймовірностей потрапляння в певний клас, (h, )
    Return:
        float: ентропія
    """

    entropy = - prob.dot(np.log2(prob))
    
    return entropy

def G(feature_column, labels):
    """
    Обчислює значення критерію G для вибраної ознаки.
    Parameters:
        feature_column (numpy.ndarray): масив значень вибраної ознаки для всіх прикладів, (N, 1)
        labels (numpy.ndarray): масив міток класів для всіх прикладів, (N, 1)
    Return:
        float: значення критерію G для вибраної ознаки
    """
    g = 0
    for value in np.unique(feature_column):
        subset_labels = labels[feature_column == value]

        _, uniq_counts = np.unique(subset_labels, return_counts=True)
        prob = uniq_counts / len(subset_labels)
        
        g += len(subset_labels) / len(labels) * Entropy(prob)
    return g


## EDA malicious and benign websites

Для цього завдання використаємо набір даних  про шкідливі та безпечні веб-сайти [malicious and benign websites](https://www.kaggle.com/datasets/xwolf12/malicious-and-benign-websites) містить 1781 записи.

Ознаки:
1. `URL-адреса`: Анонімний ідентифікатор URL.
2. `URL_LENGTH`: Кількість символів в URL.
3. `NUMBER_SPECIAL_CHARACTERS`: Кількість спеціальних символів, ідентифікованих в URL(/, %, #, &, ., =).
4. `CHARSET`: Стандарт кодування символів.
5. `SERVER`: Операційна система сервера, що обслуговує URL.
6. `CONTENT_LENGTH`: Розмір вмісту заголовка HTTP.
7. `WHOIS_COUNTRY`: Країна реєстрації URL, згідно з даними WHOIS.
8. `WHOIS_STATEPRO`: Регіон реєстрації URL, згідно з даними WHOIS.
9. `WHOIS_REGDATE`: Дата реєстрації URL, згідно з даними WHOIS.
10. `WHOIS_UPDATED_DATE`: Дата останнього оновленння URL, згідно з даними WHOIS.
11. `TCP_CONVERSATION_EXCHANGE`: Кількість TCP-пакетів, якими обмінюються сервер і honeypot.
12. `DIST_REMOTE_TCP_PORT`: Кількість портів які відрізняються від TCP-портів
13. `REMOTE_IPS`: Загальна кількість IP-адрес, підключених до honeypot.
14. `APP_BYTES`: Кількість переданих байтів між honeypot і сервером.
15. `SOURCE_APP_PACKETS`: Кількість пакетів, надісланих з honeypot на сервер.
16. `REMOTE_APP_PACKETS`: Кількість пакетів, отриманих від сервера до honeypot.
17. `APP_PACKETS`: Загальна кількість IP-пакетів, згенерованих під час зв’язку між honeypot і сервером.
18. `DNS_QUERY_TIMES`: Кількість DNS-пакетів, згенерованих під час зв’язку між honeypot і сервером.

Цільова змінна:
1. `TYPE`: тип веб-сторінки : 1 - шкідлива, 0 - безпечна

Завантажимо набір даних `dataset_Malicious_and_Benign_Websites.csv`, що знаходиться у директорії `data`,  у змінну `data1` за допомогою `pandas.read_csv`.

In [3]:
data1 = pd.read_csv("data/dataset_Malicious_and_Benign_Websites.csv")

Перші та останні 5 записів з `data1`.

In [4]:
data1

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,NaN,NaN,10/10/2015 18:21,NaN,...,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,NaN,NaN,NaN,NaN,...,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,7/10/1997 4:00,12/09/2013 0:45,...,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,NaN,124140.0,US,TX,12/05/1996 0:00,11/04/2017 0:00,...,2,5,4278,61,62,129889,4586,61,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,M4_48,194,16,UTF-8,Apache,NaN,ES,Barcelona,17/09/2008 0:00,2/09/2016 0:00,...,0,0,0,0,3,186,0,0,0.0,1
1777,M4_41,198,17,UTF-8,Apache,NaN,ES,Barcelona,17/09/2008 0:00,2/09/2016 0:00,...,0,0,0,0,2,124,0,0,0.0,1
1778,B0_162,201,34,utf-8,Apache/2.2.16 (Debian),8904.0,US,FL,15/02/1999 0:00,15/07/2015 0:00,...,2,6,6631,87,89,132181,6945,87,4.0,0
1779,B0_1152,234,34,ISO-8859-1,cloudflare-nginx,NaN,US,CA,1/04/1998 0:00,9/12/2016 0:00,...,0,0,0,0,0,0,0,0,0.0,0


Проаналізуємо кожен стовпець `data1` на наявність пропущених значень `NaN`, кількість унікальних значень та тип даних. Це потрібно для виявлення зайвих записів та стовпців, перетворення категоріальних стовпців у числові типи даних.

In [5]:
pd.DataFrame({
    "Кількість NaN": data1.notna().sum(),
    "Кількість унікальних значень": data1.nunique(),
    "Тип даних": data1.dtypes
})

,Кількість NaN,Кількість унікальних значень,Тип даних
URL,1781,1781,object
URL_LENGTH,1781,142,int64
NUMBER_SPECIAL_CHARACTERS,1781,31,int64
CHARSET,1774,8,object
SERVER,1605,238,object
CONTENT_LENGTH,969,637,float64
WHOIS_COUNTRY,1475,48,object
WHOIS_STATEPRO,1419,181,object
WHOIS_REGDATE,1654,890,object
WHOIS_UPDATED_DATE,1642,593,object


За результатами таблиці вище, видалимо наступні стовпці:
* `WHOIS_REGDATE` та `WHOIS_UPDATED_DATE` - це дати
* `CONTENT_LENGTH` - багато пропусків `NaN` 
* `URL` - має унікальні значення типу `object` для кожного запису.
* `APP_PACKETS` - сума `SOURCE_APP_PACKETS` та `REMOTE_APP_PACKETS`.

Також треба позбутися записів, що містять `NaN`, та дублікатів.

In [6]:
data1.drop(
  columns=["CONTENT_LENGTH",
           "WHOIS_REGDATE", 
           "WHOIS_UPDATED_DATE",
           "URL",
           "APP_PACKETS"], 
  inplace=True
)

data1.dropna(inplace=True)

print(f"Кількість видалених дублікатів: {data1.duplicated().sum()}")
data1.drop_duplicates(inplace=True)
data1.reset_index(drop=True,inplace=True)

Кількість видалених дублікатів: 48


Декодуємо стовпці з типом даних `object` за допомогою `LabelEncoder` та збережемо енкодери у `data/label_encoder/object_column_name.jlib`, де `object_column_name` - назва категоріального стовпця.

In [7]:
!mkdir data\label_encoder

A subdirectory or file data\label_encoder already exists.


In [8]:
object_columns = data1.select_dtypes(include=['object']).columns

for object_column_name in object_columns:
  label_encoder = LabelEncoder()
  data1[object_column_name]= label_encoder.fit_transform(data1[object_column_name])
  joblib.dump(label_encoder , f"data/label_encoder/{object_column_name}.joblib")

Збережемо очищений набір даних у `data/clear_dataset_Malicious_and_Benign_Websites.csv`.

In [9]:
data1.to_csv("data/clear_dataset_Malicious_and_Benign_Websites.csv", index=False)

## Перевірка реалізованих функцій

Спочатку перевіримо реалізацію `Entropy` за допомогою `scipy.stats.entropy`

In [10]:
prob_example = np.array([0.4, 0.3, 0.1, 0.2])

print(f"Entropy({prob_example}) = {Entropy(prob_example):.3f}")
print(f"entropy({prob_example}) = {entropy(prob_example , base=2):.3f}")


Entropy([0.4 0.3 0.1 0.2]) = 1.846
entropy([0.4 0.3 0.1 0.2]) = 1.846


Порахуємо залишкову ентропію для кожної нецільової змінної:

In [11]:
for i, object_column_name in enumerate(object_columns):
  print(f"G({object_column_name }) = {G(data1[object_column_name ], data1["Type"]):.3f}")

G(CHARSET) = 0.440
G(SERVER) = 0.296
G(WHOIS_COUNTRY) = 0.260
G(WHOIS_STATEPRO) = 0.133


## Висновки

Було реалізовано функції згідно з варіантами, та протестовано їх на реальному наборі даних, який перед цим обробили та зберегли для подальшого використання.